In [ ]:
import sys, os, argparse
import pandas as pd
import warnings

## load functions from utils.py
from utils import load_dataset, train_model, get_consensus_signatures, encoder_prediction

warnings.filterwarnings("ignore")


## pass arguments
parser = argparse.ArgumentParser()
parser.add_argument('-r', '--filename_real_data', type=str, required=True, help='Real dataset (i.e. before resamplings), all samples')
parser.add_argument('-t', '--filename_permuted_data_training', type=str, required=True, help='PERMUTED TRAINING samples multiple tables (output from 1_parse_input.R; .tsv)')
parser.add_argument('-v', '--filename_permuted_data_validation', type=str, required=True, help='PERMUTED VALIDATION samples single table (output from 1_parse_input.R; .tsv)')
parser.add_argument('-k', '--n_signatures', type=int, help='Number of signatures (neurons in the latent space) to explore', default = 2)
parser.add_argument('--iters', type=int, default=100, help='Number of independent trainings; then, all the k×iters signatures generated are clustered via K-means into k clusters, and the consensus k signatures are returned')
parser.add_argument('--epochs', type=int, default=1000, help='Number of epochs; WARNING: there needs to exist in ./documents/ 1 different resampled training table per epoch, with the training filenames reflecting this, i.e. ./documents/*iter[1:epochs]*')
parser.add_argument('--batch_size', type=int, default=64, help='Batch size')
parser.add_argument('--l1_size', type=int, default=128, help='N neurons of the first encoder layer; the 2nd and 3rd are fractions thereof')
parser.add_argument('--validation_perc', type=int, default=20, help='% of the total samples that is reserved for validation; this is done a priori with 1_parse_input.R')
parser.add_argument('--loss', type=str, default='mean_squared_error', help='Loss function to use in the autoencoder')
parser.add_argument('--activation', type=str, default='softplus', help='Activation function')
parser.add_argument('--normalization', type=str, default='yes', help='Whether or not to perform standardization of the input layer (coefficients)')
parser.add_argument('--allow_negative_weights', type=str, default='yes', help='If yes (default), negative weights in the decoder layer are allowed. Otherwise, a non-negative constraint is applied to the decoder layer weights')
parser.add_argument('--inputDir', type=str, default='$PWD/datasets/', help='Directory where input data is stored')
parser.add_argument('--outputDir', type=str, default='$PWD/res/', help='Directory to save results')
parser.add_argument('--seed', type=int, required=False, help='Specify a seed, otherwise it will be selected based on the current time')

if 'ipykernel' in sys.modules: # if interactive, pass values manually
    filename_real_data = "original_coeff.tsv"
    filename_permuted_data_training = "perm_coeff_iter*_training.tsv"
    filename_permuted_data_validation = "perm_coeff_validation.tsv"
    n_signatures = 2
    iters = 2
    epochs = 100
    batch_size = 64
    l1_size = 128
    validation_perc = 10
    loss = 'mean_squared_error'
    activation = 'softplus'
    normalization = 'no'
    allow_negative_weights = 'yes'
    inputDir = "./datasets/"
    outputDir = "./res/"
else:
    args = parser.parse_args()
    filename_real_data = args.filename_real_data
    filename_permuted_data_training = args.filename_permuted_data_training
    filename_permuted_data_validation = args.filename_permuted_data_validation
    n_signatures = args.n_signatures
    iters = args.iters
    epochs = args.epochs
    batch_size = args.batch_size
    l1_size = args.l1_size
    validation_perc = args.validation_perc
    loss = args.loss
    activation = args.activation
    normalization = args.normalization
    allow_negative_weights = args.allow_negative_weights
    inputDir = args.inputDir
    outputDir = args.outputDir

    
## load datasets
# they have to be in ./inputDir/validation_perc_{validation_perc}/
# n_features indicates the number of input neurons (1 per feature included in regressions)
# there must be as many available training files as epochs are specified
real_data_df,real_data_sample_names,feature_names,n_features,training_validation_dfs_dict,seed = load_dataset(filename_real_data,
                                                                                                              filename_permuted_data_training,
                                                                                                              filename_permuted_data_validation,
                                                                                                              epochs,
                                                                                                              validation_perc,
                                                                                                              normalization,
                                                                                                              inputDir,
                                                                                                              seed=None)
## create output folder
output_folder_name = f'nFeatures_{str(n_features)}__' \
                     f'nSignatures_{str(n_signatures)}__' \
                     f'nIters_{str(iters)}__' \
                     f'nEpochs_{str(epochs)}__' \
                     f'batchSize_{str(batch_size)}__' \
                     f'l1Size_{str(l1_size)}__' \
                     f'validationPerc_{str(validation_perc)}__' \
                     f'normalization_{str(normalization)}__' \
                     f'allow_negative_weights_{str(allow_negative_weights)}__' \
                     f'seed_{str(seed)}/'

if 'ipykernel' in sys.modules: # if interactive, create folders
    if not os.path.exists(outputDir):
        os.mkdir(outputDir)
    output_folder_name = outputDir + output_folder_name
    if not os.path.exists(output_folder_name):
        os.mkdir(output_folder_name)
else: ## not interactive (nextflow handles the 'outputDir/' folder creation)
    os.mkdir(output_folder_name)


## run training with simultaneous validation

# >=1 iterations in parallel
results = {}
for i in range(iters):
    seed = seed+i
    results[i] = train_model(training_validation_dfs_dict,
                             n_features,
                             feature_names,
                             n_signatures,
                             epochs,
                             batch_size,
                             l1_size,
                             loss,
                             activation,
                             allow_negative_weights,
                             seed,
                             output_folder_name,
                             i)
    
extractions = []
best_model_losses_epoch_alliters = pd.DataFrame({'output_folder_name': [],
                                                 'iter': [],
                                                 'seed': [],
                                                 'best_model_training_loss': [],
                                                 'best_model_validation_loss': [],
                                                 'min_val_loss_epoch': []})
for i in range(iters):
    # get all K×iters signatures (without feature names column)
    extractions.append(results[i][3].drop('Feature', axis=1))
    # get also the best_model_losses_epoch_i for each iteration
    best_model_losses_epoch_alliters = pd.concat([best_model_losses_epoch_alliters, results[i][2]], axis=0)
    
# store feature names column
feature_names_col = results[0][3].loc[:, 'Feature']

# get as an example of a loss plot the one from the first iteration
loss_plot_first_iter = results[0][1]


## cluster all K×iters signatures into K clusters and get the K consensus signatures
min_sil, mean_sil, consensus_sig, means_lst = get_consensus_signatures(n_signatures, extractions, feature_names_col)


## Refitting: fix the decoder on the consensus signatures, and then train the encoder on the the original (i.e. not resampled) coefficients matrix, to get the sample exposures (latent representation)
trained_encoder, encoded_real_df = encoder_prediction(real_data_df=real_data_df,
                                                      real_data_sample_names=real_data_sample_names,
                                                      S=consensus_sig,
                                                      input_dim=n_features,
                                                      l1_size=l1_size,
                                                      n_signatures=n_signatures,
                                                      batch_size=batch_size,
                                                      allow_negative_weights=allow_negative_weights,
                                                      seed=seed,
                                                      output_folder_name=output_folder_name)

## save outputs

# training performance plot
loss_plot_first_iter.savefig(output_folder_name + 'loss_plot_first_iter.jpg', dpi=100)    

# save best_model_losses_epoch summary (for all iterations) outside the output folder, nextflow will concatenate all 
best_model_losses_epoch_alliters.to_csv('best_model_losses_epoch_alliters.tsv', sep='\t', index= False)

# best encoder model 
trained_encoder.save(output_folder_name + 'best_encoder_model.tf')

# encoded layer ("signature exposures")
encoded_real_df.to_csv(output_folder_name + 'signature_exposures.tsv', sep='\t', index= False)

# decoder layer weights (consensus "signature weights")
consensus_sig.to_csv(output_folder_name + 'signature_weights.tsv', sep='\t', index= False)